In [ ]:
import pandas as pd
import numpy as np
from array import array
import ROOT
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [ ]:
def FitFunction(x, a, b):
    return a*x+b

In [ ]:
def GetDataArrays (df):
    voltage = df.Tensione
    voltage_err = df.errore_tensione
    channel = df.media
    channel_err = df.errore
    
    return voltage, voltage_err, channel, channel_err

In [ ]:
def PlotMCAConversion(df):
    voltage, voltage_err, channel, channel_err = GetDataArrays(df)
    
    plt.errorbar(voltage, channel, xerr=voltage_err, yerr=channel_err, fmt=".r")
    plt.title("MCA Conversion Linearity")
    plt.xlabel("Tension [V]")
    plt.ylabel("Channel Number")
    plt.grid(True)

In [ ]:
def LinearFit(df):
    voltage = df.Tensione
    channel = df.media
    
    par, pcov = curve_fit(FitFunction, voltage, channel)
    return par, pcov

# MCA conversion linearity

We had some issues with the MCA conversion linearity measurements: instead of having a single peak centered corrisponding to a given pulse, we kept having a double peak. We tried changing amplifier, then we changed pulser and the double peak kept showing up. We thus started removing modules from the crate and the lowest of the two peaks started to decrease. We interpreted it as a noise given by the crate and fitted the highest peak, the lowest one and finally both peaks with a Gaussian.

In [ ]:
# load data; we need to convert files from .ods to .xlsx
xls = pd.ExcelFile("./dati/Dati_line.xlsx")
# dataframe for each of the three casese (header00 is set to avoid titles)
df1 = pd.read_excel(xls, 'picco grande', header=0)
df2 = pd.read_excel(xls, 'picco piccolo', header=0)
df3 = pd.read_excel(xls, 'centroide', header=0)

## Highest peak

In [ ]:
PlotMCAConversion(df1)

In [ ]:
par, pcov = LinearFit(df1)

In [ ]:
diff = df1.media-FitFunction(df1.Tensione, *par)
diff = diff/FitFunction(df1.Tensione, *par)
plt.errorbar(df1.Tensione, diff,yerr=df1.errore/FitFunction(df1.Tensione, *par), fmt=".r")
plt.axhline(y=0)
plt.xlabel("Voltage [V]")
plt.ylabel("Residue")
plt.grid(True)

## Lowest peak

In [ ]:
PlotMCAConversion(df2)

In [ ]:
par, pcov = LinearFit(df2)

In [ ]:
diff = df2.media-FitFunction(df2.Tensione, *par)
diff = diff/FitFunction(df2.Tensione, *par)
plt.errorbar(df2.Tensione, diff,yerr=df2.errore/FitFunction(df2.Tensione, *par), fmt=".r")
plt.axhline(y=0)
plt.xlabel("Voltage [V]")
plt.ylabel("Residue")
plt.grid(True)

## Centroid

In [ ]:
PlotMCAConversion(df2)

In [ ]:
par, pcov = LinearFit(df2)

In [ ]:
diff = df3.media-FitFunction(df3.Tensione, *par)
diff = diff/FitFunction(df3.Tensione, *par)
plt.errorbar(df3.Tensione, diff,yerr=df3.errore/FitFunction(df3.Tensione, *par), fmt=".r")
plt.axhline(y=0)
plt.xlabel("Voltage [V]")
plt.ylabel("Residue")
plt.grid(True)

## ROOT version of the script

In [ ]:
voltage = (df1['Tensione']).to_numpy()
channel = (df1['media']).to_numpy()
errore_media = (df1['errore']).to_numpy()
errore_tensione = (df1['errore_tensione']).to_numpy()
n = np.size(channel)

In [ ]:
graph = ROOT.TGraphErrors(n, voltage, channel, errore_tensione, errore_media)
cnv = ROOT.TCanvas("", "", 800, 700)

In [ ]:
graph.GetXaxis().SetTitle("Voltage [V]")
graph.GetYaxis().SetTitle("Channel")
graph.SetMarkerStyle(20)
graph.Draw("AP")

In [ ]:
cnv.Draw()